In [1]:
#import the libraries 
import requests
import urllib
import psycopg2
import os

#### Connect to ```database``` to get the movies title

In [2]:
try:
    connection = psycopg2.connect(user = 'postgres',
                                 password = '0000!',
                                 host = '127.0.0.1',
                                 port = '5432',
                                 database = 'disney_movies')
    cur = connection.cursor()
    
except:
    print("ERROR_CONNECTING_DATABASE")

#### Get the ```title```

In [3]:
cur.execute('''SELECT title FROM movies_table''')
titles = cur.fetchall()
type(titles)

list

In [9]:
for title in titles:
    print(title[0])
    if title[0] == 'Around the World in 80 Days':
        print()
        print()

The Jungle Book 2
Academy Award Review of Walt Disney Cartoons
Snow White and the Seven Dwarfs
Pinocchio
Fantasia
The Reluctant Dragon
Dumbo
Dumbo
Bambi
Saludos Amigos
Victory Through Air Power
The Three Caballeros
Make Mine Music
Song of the South
Fun and Fancy Free
Melody Time
So Dear to My Heart
The Adventures of Ichabod and Mr. Toad
Cinderella
Cinderella
Treasure Island
Alice in Wonderland
Alice in Wonderland
The Story of Robin Hood and His Merrie Men
Peter Pan
The Sword and the Rose
The Living Desert
Rob Roy, the Highland Rogue
The Vanishing Prairie
20,000 Leagues Under the Sea
Davy Crockett, King of the Wild Frontier
Lady and the Tramp
The African Lion
The Littlest Outlaw
The Great Locomotive Chase
Davy Crockett and the River Pirates
Secrets of Life
Westward Ho the Wagons!
Johnny Tremain
Perri
Old Yeller
The Light in the Forest
White Wilderness
Tonka
Sleeping Beauty
The Shaggy Dog
The Shaggy Dog
Darby O'Gill and the Little People
Third Man on the Mountain
Toby Tyler
Kidnapped
Pol

In [10]:
base_url = 'http://www.omdbapi.com/?'
def get_data_from_OMDb(title):
    parameters = {'t':title, 'apikey':os.environ['OMDb_API_KEY']}
    full_url = base_url + urllib.parse.urlencode(parameters)
    return requests.get(full_url).json()

In [ ]:
title = 'America's Heart and Soul'
get_data_from_OMDb('America\'s Heart and Soul')

In [5]:
res = get_data_from_OMDb('The Jungle Book')
print(res)

{'Title': 'The Jungle Book', 'Year': '2016', 'Rated': 'PG', 'Released': '15 Apr 2016', 'Runtime': '106 min', 'Genre': 'Adventure, Drama, Family, Fantasy', 'Director': 'Jon Favreau', 'Writer': 'Justin Marks (screenplay by), Rudyard Kipling (based on the books by)', 'Actors': 'Neel Sethi, Bill Murray, Ben Kingsley, Idris Elba', 'Plot': 'After a threat from the tiger Shere Khan forces him to flee the jungle, a man-cub named Mowgli embarks on a journey of self discovery with the help of panther Bagheera and free-spirited bear Baloo.', 'Language': 'English', 'Country': 'UK, USA', 'Awards': 'Won 1 Oscar. Another 32 wins & 55 nominations.', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTc3NTUzNTI4MV5BMl5BanBnXkFtZTgwNjU0NjU5NzE@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '94%'}, {'Source': 'Metacritic', 'Value': '77/100'}], 'Metascore': '77', 'imdbRating': '7.4', 'imdbVotes': '255,790', 'imdbID': 'tt304096

In [11]:
create_omdb_data = '''CREATE TABLE omdb(
                        id BIGSERIAL NOT NULL,
                        Title TEXT,
                        Genre TEXT,
                        Metascore VARCHAR(20),
                        imdbRating DECIMAL,
                        imdbVotes VARCHAR(20),
                        imdbID VARCHAR(25),
                        rotten_Tomatoes VARCHAR(10));'''
cur.execute(create_omdb_data)
connection.commit()

In [12]:
def get_rotten_tomatoes_score(movie_info):
    ratings = movie_info.get('Ratings',[])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
        return None

In [13]:
print(get_rotten_tomatoes_score(res))

None


In [14]:
index = 1
total = len(titles)
for title in titles:
    try:
        
        movie_info = get_data_from_OMDb(title)
        rotten_Tomatoes = get_rotten_tomatoes_score(movie_info)
        insert_query = '''
                            INSERT INTO omdb(Title, Genre, Metascore, imdbRating, imdbVotes, imdbID, rotten_Tomatoes)
                            VALUES(%s, %s, %s, %s, %s, %s, %s)'''
        record_to_insert = (movie_info['Title'], movie_info['Genre'], movie_info['Metascore'], movie_info['imdbRating'], movie_info['imdbVotes'], movie_info['imdbID'], rotten_Tomatoes)

        # insert into database
        cur.execute(insert_query, record_to_insert)
        connection.commit()
        print(index, total, "-----", movie_info['Title'])
        index += 1
        
    except:
        print('00000Xa0000000000000 ERROR')

1 341 ----- The Jungle Book 2
2 341 ----- Academy Award Review of Walt Disney Cartoons
3 341 ----- Snow White and the Seven Dwarfs
4 341 ----- Pinocchio
5 341 ----- Fantasia
6 341 ----- The Reluctant Dragon
7 341 ----- Dumbo
8 341 ----- Dumbo
9 341 ----- Bambi
10 341 ----- Saludos Amigos
11 341 ----- Victory Through Air Power
12 341 ----- The Three Caballeros
13 341 ----- Make Mine Music
14 341 ----- Song of the South
15 341 ----- Fun and Fancy Free
16 341 ----- Melody Time
17 341 ----- So Dear to My Heart
18 341 ----- The Adventures of Ichabod and Mr. Toad
19 341 ----- Cinderella Man
20 341 ----- Cinderella Man
21 341 ----- Muppet Treasure Island
22 341 ----- Alice in Wonderland
23 341 ----- Alice in Wonderland
24 341 ----- The Story of Robin Hood and His Merrie Men
25 341 ----- Peter Pan
26 341 ----- The Sword and the Rose
27 341 ----- The Living Desert
28 341 ----- Rob Roy: The Highland Rogue
29 341 ----- The Vanishing Prairie
30 341 ----- 20,000 Leagues Under the Sea
31 341 ----- D

240 341 ----- College Road Trip
241 341 ----- The Chronicles of Narnia: Prince Caspian
242 341 ----- WALL·E
243 341 ----- Beverly Hills Chihuahua
244 341 ----- Morning Light
00000Xa0000000000000 ERROR
245 341 ----- Bolt
246 341 ----- Bedtime Stories
247 341 ----- Jonas Brothers: The 3D Concert Experience
248 341 ----- Race to Witch Mountain
249 341 ----- Hannah Montana: The Movie
250 341 ----- Up
251 341 ----- Lilly the Witch: The Dragon and the Magic Book
252 341 ----- G-Force
253 341 ----- Walt & El Grupo
254 341 ----- Old Dogs
255 341 ----- The Princess and the Frog
256 341 ----- Alice in Wonderland
257 341 ----- Alice in Wonderland
258 341 ----- Prince of Persia: The Sands of Time
259 341 ----- Toy Story 3
260 341 ----- The Sorcerer's Apprentice
261 341 ----- The Crimson Wing: Mystery of the Flamingos
262 341 ----- Secretariat
263 341 ----- Do Dooni Chaar
264 341 ----- Tangled
265 341 ----- The Boys: The Sherman Brothers' Story
266 341 ----- TRON: Legacy
00000Xa0000000000000 ERROR


In [45]:
if connection:
    cur.close()
    connection.close()
    print("Connection Closed!")

Connection Closed!
